# #MakeoverMonday 2019w32: Coal Consumption in the UK

#### The data analysed below was downloaded from [Gridwatch UK](https://gridwatch.co.uk/). It provides daily energy production figures by source from 2012 to 2019.

First, I import the libraries I'll need.

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Then, I import the data and run some high-level descriptive analytics on it.

In [25]:
uk_energy = pd.read_csv('gridwatch.csv', parse_dates=True)

In [26]:
uk_energy.shape

(796453, 12)

In [27]:
uk_energy.head()

,id,timestamp,demand,frequency,coal,nuclear,ccgt,wind,pumped,hydro,biomass,solar
0,62694,2012-01-01 00:00:01,30590,50.126999,8693,7121,8568,2740,0,728,0,0.0
1,62695,2012-01-01 00:05:06,30490,50.039001,8650,7120,8441,2812,0,732,0,0.0
2,62696,2012-01-01 00:10:01,30802,50.000000,8880,7125,8427,2896,0,744,0,0.0
3,62697,2012-01-01 00:15:01,31180,50.028999,9111,7122,8494,2964,0,748,0,0.0
4,62698,2012-01-01 00:20:01,31241,50.002998,9195,7114,8449,2992,0,750,0,0.0


In [28]:
uk_energy.describe()

,id,demand,frequency,coal,nuclear,ccgt,wind,pumped,hydro,biomass,solar
count,796453.000000,796453.000000,796453.000000,796453.000000,796453.000000,796453.000000,796453.000000,796453.000000,796453.000000,796453.000000,7.964530e+05
mean,460920.903601,33441.390026,49.949458,7533.566650,7214.377451,11540.665320,2905.132293,303.329773,400.898907,1186.621983,4.413354e+02
std,229916.472183,7238.845210,1.609492,6410.003369,934.940076,5166.125208,2233.491732,328.883426,255.525493,745.151475,2.587864e+03
min,62694.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,261808.000000,27874.000000,49.959999,1226.000000,6684.000000,7361.000000,1142.000000,0.000000,189.000000,679.000000,0.000000e+00
50%,460921.000000,32737.000000,49.997002,6420.000000,7337.000000,11352.000000,2353.000000,290.000000,362.000000,1116.000000,0.000000e+00
75%,660034.000000,38495.000000,50.042000,13070.000000,7888.000000,15167.000000,4162.000000,437.000000,575.000000,1918.000000,0.000000e+00
max,859147.000000,254852.000000,50.403000,25239.000000,9346.000000,27150.000000,12456.000000,2861.000000,1434.000000,3093.000000,1.420000e+06


In [29]:
uk_energy.columns

Index(['id', ' timestamp', ' demand', ' frequency', ' coal', ' nuclear',
       ' ccgt', ' wind', ' pumped', ' hydro', ' biomass', ' solar'],
      dtype='object')

The column names are already in a format that's easy to use. There seems to be some spaces in the titles though that should be cleaned.

In [30]:
uk_energy.columns = [c.strip() for c in uk_energy.columns]

In [31]:
uk_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796453 entries, 0 to 796452
Data columns (total 12 columns):
id           796453 non-null int64
timestamp    796453 non-null object
demand       796453 non-null int64
frequency    796453 non-null float64
coal         796453 non-null int64
nuclear      796453 non-null int64
ccgt         796453 non-null int64
wind         796453 non-null int64
pumped       796453 non-null int64
hydro        796453 non-null int64
biomass      796453 non-null int64
solar        796453 non-null float64
dtypes: float64(2), int64(9), object(1)
memory usage: 72.9+ MB


The `timestamp` column is not of date type, let's change that.

In [32]:
uk_energy['timestamp'] = pd.to_datetime(uk_energy['timestamp'])

There are many data points for each day. I'm going to resample the dataframe to have a single row for a given day.

In [33]:
uk_energy_daily = uk_energy.set_index('timestamp')

In [34]:
uk_energy_daily.head()

,id,demand,frequency,coal,nuclear,ccgt,wind,pumped,hydro,biomass,solar
timestamp,,,,,,,,,,,
2012-01-01 00:00:01,62694,30590,50.126999,8693,7121,8568,2740,0,728,0,0.0
2012-01-01 00:05:06,62695,30490,50.039001,8650,7120,8441,2812,0,732,0,0.0
2012-01-01 00:10:01,62696,30802,50.000000,8880,7125,8427,2896,0,744,0,0.0
2012-01-01 00:15:01,62697,31180,50.028999,9111,7122,8494,2964,0,748,0,0.0
2012-01-01 00:20:01,62698,31241,50.002998,9195,7114,8449,2992,0,750,0,0.0


In [35]:
uk_energy_daily = uk_energy_daily.resample('D').mean()

In [36]:
uk_energy_daily.head()

,id,demand,frequency,coal,nuclear,ccgt,wind,pumped,hydro,biomass,solar
timestamp,,,,,,,,,,,
2012-01-01,62837.5,31914.652778,49.990587,8501.128472,7047.003472,9822.274306,2640.236111,394.107639,786.402778,0.0,0.0
2012-01-02,63125.0,34677.912892,49.998080,10203.655052,7075.986063,11222.003484,2762.261324,326.686411,777.780488,0.0,0.0
2012-01-03,63412.5,40226.892361,50.007969,15384.989583,6964.746528,12183.840278,2159.430556,325.079861,804.246528,0.0,0.0
2012-01-04,63700.5,41752.642361,49.984062,17026.888889,7079.906250,11425.777778,2654.190972,370.715278,806.322917,0.0,0.0
2012-01-05,63988.5,41370.364583,50.007097,16545.250000,7540.791667,10884.895833,2639.517361,375.489583,827.434028,0.0,0.0


In [37]:
uk_energy_daily.shape

(2772, 11)

In [38]:
uk_energy_daily.reset_index(inplace=True)
uk_energy_daily.head()

,timestamp,id,demand,frequency,coal,nuclear,ccgt,wind,pumped,hydro,biomass,solar
0,2012-01-01,62837.5,31914.652778,49.990587,8501.128472,7047.003472,9822.274306,2640.236111,394.107639,786.402778,0.0,0.0
1,2012-01-02,63125.0,34677.912892,49.998080,10203.655052,7075.986063,11222.003484,2762.261324,326.686411,777.780488,0.0,0.0
2,2012-01-03,63412.5,40226.892361,50.007969,15384.989583,6964.746528,12183.840278,2159.430556,325.079861,804.246528,0.0,0.0
3,2012-01-04,63700.5,41752.642361,49.984062,17026.888889,7079.906250,11425.777778,2654.190972,370.715278,806.322917,0.0,0.0
4,2012-01-05,63988.5,41370.364583,50.007097,16545.250000,7540.791667,10884.895833,2639.517361,375.489583,827.434028,0.0,0.0


Now, I want to know the daily % share of each energy source. To do so, I'm going to first calculate the `total_output`.

In [39]:
uk_energy_daily['total_output'] = uk_energy_daily['coal'] + uk_energy_daily['nuclear'] +\
                                  uk_energy_daily['ccgt'] + uk_energy_daily['wind'] +\
                                  uk_energy_daily['pumped'] + uk_energy_daily['hydro'] +\
                                  uk_energy_daily['biomass'] + uk_energy_daily['solar']

In [40]:
uk_energy_daily.head()

,timestamp,id,demand,frequency,coal,nuclear,ccgt,wind,pumped,hydro,biomass,solar,total_output
0,2012-01-01,62837.5,31914.652778,49.990587,8501.128472,7047.003472,9822.274306,2640.236111,394.107639,786.402778,0.0,0.0,29191.152778
1,2012-01-02,63125.0,34677.912892,49.998080,10203.655052,7075.986063,11222.003484,2762.261324,326.686411,777.780488,0.0,0.0,32368.372822
2,2012-01-03,63412.5,40226.892361,50.007969,15384.989583,6964.746528,12183.840278,2159.430556,325.079861,804.246528,0.0,0.0,37822.333333
3,2012-01-04,63700.5,41752.642361,49.984062,17026.888889,7079.906250,11425.777778,2654.190972,370.715278,806.322917,0.0,0.0,39363.802083
4,2012-01-05,63988.5,41370.364583,50.007097,16545.250000,7540.791667,10884.895833,2639.517361,375.489583,827.434028,0.0,0.0,38813.378472


In [41]:
uk_energy_daily['perc_coal'] = uk_energy_daily['coal'] / uk_energy_daily['total_output']
uk_energy_daily['perc_nuclear'] = uk_energy_daily['nuclear'] / uk_energy_daily['total_output']
uk_energy_daily['perc_ccgt'] = uk_energy_daily['ccgt'] / uk_energy_daily['total_output']
uk_energy_daily['perc_wind'] = uk_energy_daily['wind'] / uk_energy_daily['total_output']
uk_energy_daily['perc_pumped'] = uk_energy_daily['pumped'] / uk_energy_daily['total_output']
uk_energy_daily['perc_hydro'] = uk_energy_daily['hydro'] / uk_energy_daily['total_output']
uk_energy_daily['perc_biomass'] = uk_energy_daily['biomass'] / uk_energy_daily['total_output']
uk_energy_daily['perc_solar'] = uk_energy_daily['solar'] / uk_energy_daily['total_output']

The renewables represent a small percentage share if looked at separately; it would be better to group them.

In [42]:
uk_energy_daily['perc_renewables'] = uk_energy_daily['perc_wind'] + uk_energy_daily['perc_biomass']\
+ uk_energy_daily['perc_solar'] + uk_energy_daily['perc_hydro'] + uk_energy_daily['perc_pumped']

In [43]:
uk_energy_daily.tail()

,timestamp,id,demand,frequency,coal,nuclear,ccgt,wind,pumped,hydro,...,total_output,perc_coal,perc_nuclear,perc_ccgt,perc_wind,perc_pumped,perc_hydro,perc_biomass,perc_solar,perc_renewables
2767,2019-07-30,857896.5,27818.041667,50.010385,0.0,5465.274306,12729.458333,4283.743056,96.697917,255.684028,...,26438.181719,0.0,0.206719,0.481480,0.162029,0.003658,0.009671,0.087005,0.049439,0.311801
2768,2019-07-31,858184.5,27409.159722,49.994785,0.0,5579.392361,10936.500000,5852.725694,80.784722,322.725694,...,26305.752653,0.0,0.212098,0.415746,0.222488,0.003071,0.012268,0.087419,0.046910,0.372157
2769,2019-08-01,858472.5,27888.045139,50.009997,0.0,5579.652778,14669.347222,2258.722222,141.277778,406.680556,...,27167.640913,0.0,0.205379,0.539957,0.083140,0.005200,0.014969,0.084981,0.066374,0.254665
2770,2019-08-02,858760.5,27531.281250,49.997410,0.0,5528.402778,15336.781250,1027.447917,220.038194,400.465278,...,26864.965625,0.0,0.205785,0.570884,0.038245,0.008191,0.014907,0.084920,0.077069,0.223331
2771,2019-08-03,859026.0,24489.448560,49.996078,0.0,5442.181070,12787.181070,640.621399,166.670782,313.300412,...,23175.791165,0.0,0.234822,0.551747,0.027642,0.007192,0.013518,0.074097,0.090982,0.213431


Finally, I'm going to melt the dataframe into a Tableau-friendly format and export it.

In [44]:
uk_energy_daily_melted = pd.melt(uk_energy_daily, 'timestamp', value_vars=['perc_coal', 
    'perc_nuclear', 'perc_ccgt', 'perc_wind', 'perc_pumped',
    'perc_hydro', 'perc_biomass', 'perc_solar', 'perc_renewables'])

In [45]:
uk_energy_daily_melted.sample(10)

,timestamp,variable,value
19707,2012-10-30,perc_solar,0.000000
22638,2013-04-07,perc_renewables,0.044696
21796,2018-07-20,perc_solar,0.061953
18497,2017-02-08,perc_biomass,0.051517
19140,2018-11-13,perc_biomass,0.054380
6289,2014-01-15,perc_ccgt,0.251222
12886,2016-12-03,perc_pumped,0.006892
11286,2012-07-17,perc_pumped,0.010887
15808,2017-05-02,perc_hydro,0.006254
18108,2016-01-16,perc_biomass,0.059415


In [46]:
uk_energy_daily_melted.to_csv('gridwatch_resampled_melted.csv', index=False)